In [1]:
import numpy as np

r = np.random.beta(0.1, 1)
print r

1.99104426988e-11


In [2]:
from xml.etree import cElementTree as ET
import sys

posts = open('Posts.xml', 'r').read()

posts[1:100]

def remove_tags(text):
    return ''.join(ET.fromstring(text).itertext())

root = ET.fromstring(posts)
documents = []
for child in root.findall('row')[0:100]:
    text = None
    try:
        child_text = child.get('Body').encode('utf-8').strip()
        text = remove_tags(child_text)
    except ET.ParseError as e:
        #print("Caught exception: " + str(e))
        "Caught exception: " + str(e)
    if text != None: documents.append(text)
        
    
print documents[0:10]

['I was offered a beer the other day that was reportedly made with citra hops. What are citra hops? Why should I care that my beer is made with them?', 'As far as we know, when did humans first brew beer, and where? Around when would you have been able to get your hands on something resembling a modern lager?', "How is low/no alcohol beer made? I'm assuming that the beer is made normally and the alcohol is then removed, is it any more than just boiling it off? I've noticed that no/low alcohol beers' taste improved hugely a few years ago, is this due to a new technique?", "In general, what's the best way to work out the temperature at which to serve a particular beer? Room temperature? Cold? Supercold? Warm?", "Assuming we're comparing equivalent amounts of alcohol, do certain beers get you inebriated more quickly or slowly? Does the amount of fizz make a difference?", 'Apart from coming out of different taps, some ales seem very similar to lagers (although there are clearly a much grea

In [3]:
execfile('corpus.py')
execfile('document.py')

corpus = Corpus(documents, '../Week1HW/stopwords.txt', 3)

print corpus.docs[0].tokens
# FIXME: Not sure why this is not 100
print corpus.N

[u'wa' u'offer' u'beer' u'the' u'day' u'wa' u'reportedli' u'citra' u'hop'
 u'are' u'citra' u'hop' u'whi' u'care' u'beer']
40


In [37]:
# Initizations
#
D = len(corpus.docs)
K = 3 # K - numer of topics
V = len(corpus.token_set)

# Randomly assign each word in each document to a topic
# each array is of "arbitrary" length - length of document's respective word list
word_topic_assignments = [[]]*corpus.N
for doci, doc in enumerate(corpus.docs):
    word_topic_assignments[doci] = []
    for wordi, word in enumerate(doc.tokens):
        word_topic_assignments[doci].append(np.random.randint(0,K))

print word_topic_assignments[0]

# Initialize document counts of words for each topic, D x K
document_topic_word_distribution = np.zeros((D, K))
for doci, doc in enumerate(corpus.docs):
    # sum the number of words in topic k
    for k in range(0,K):
        document_topic_word_distribution.itemset((doci, k), word_topic_assignments[doci].count(k))

print document_topic_word_distribution[0,:]

# Initialize term topic counts, e.g. number of times term V was allocated to topic K, K x V
# For every word in every document, determine what it's term index is
topic_term_distribution = np.zeros((K, V))
termlist = list(corpus.token_set)
for doci, doc in enumerate(corpus.docs):
    for wordi, word in enumerate(doc.tokens):
        termidx = termlist.index(word)
        topic_alloc = word_topic_assignments[doci][wordi]
        current_count = topic_term_distribution.item((topic_alloc,termidx))
        topic_term_distribution.itemset((topic_alloc,termidx), current_count + 1)
        
print topic_term_distribution[0,:][0:10]

alpha = 50/K
eta = 200/V
iters = 10

terms_to_sanity_check = ['skunk', 'can', 'wine', 'saison']
for term in terms_to_sanity_check:
    termidx = termlist.index(term)    
    print 'Term: ' + term
    print 'Original topic distribution: ' + str(topic_term_distribution[:,termidx])
    

for i in range(iters):
    if i%25 == 0: print 'Iteration ' + str(i)
    # 1 iteration
    for doci, doc in enumerate(corpus.docs):
        for wordi, word in enumerate(doc.tokens):
            word_topic_alloc = word_topic_assignments[doci][wordi]
            termidx = termlist.index(word)
            # decrement counts
            current_doc_count = document_topic_word_distribution.item((doci, word_topic_alloc))
            document_topic_word_distribution.itemset((doci, word_topic_alloc), current_doc_count - 1)
            current_topic_count = topic_term_distribution.item((word_topic_alloc, termidx))
            topic_term_distribution.itemset((word_topic_alloc, termidx), current_topic_count - 1)
            # update theta for each document based on new topic allocation
            # draw theta - Dirichlet with paramters (alpha + n_{d,k}) - num of words in doc with topic alloc k
            theta_params = alpha + document_topic_word_distribution[doci,:] + 1
            # theta for this document
            theta = np.random.dirichlet(tuple(theta_params), 1)
            # draw beta - Dirichlet with paramters (eta + m_{k,v}) - num of times term appeared in topic k
            beta_params = eta + topic_term_distribution[word_topic_alloc,:] + 1
            beta = np.random.dirichlet(tuple(beta_params), 1)
            probs = theta[0]*beta[0][termidx]/np.sum(theta[0]*beta[0][termidx])
            new_topic_alloc = np.random.multinomial(1, tuple(probs))
            new_topic_alloc = list(new_topic_alloc).index(1)                
            word_topic_assignments[doci][wordi] = new_topic_alloc
            # increment counts
            current_doc_count = document_topic_word_distribution.item((doci, new_topic_alloc))
            document_topic_word_distribution.itemset((doci, new_topic_alloc), current_doc_count + 1)
            current_topic_count = topic_term_distribution.item((new_topic_alloc, termidx))
            topic_term_distribution.itemset((new_topic_alloc, termidx), current_topic_count + 1)            

# FIXME: I think we need to take the draws of beta and theta out of the loop for words
# In other words, loop through all the words using an initial value of beta and theta
# and then update in a separate step with random draws based on new counts

for termi, term in enumerate(terms_to_sanity_check):
    termidx = termlist.index(term)
    print 'Term: ' + term
    print 'New topic distribution: ' + str(topic_term_distribution[:,termidx])


[2, 1, 0, 1, 0, 1, 0, 2, 0, 2, 0, 1, 1, 0, 2]
[ 6.  5.  4.]
[ 0.  0.  2.  1.  1.  0.  1.  1.  1.  0.]
Term: skunk
Original topic distribution: [ 2.  2.  4.]
Term: can
Original topic distribution: [ 3.  2.  4.]
Term: wine
Original topic distribution: [ 2.  1.  1.]
Term: saison
Original topic distribution: [ 0.  3.  1.]
Iteration 0
[ 0.25890958  0.40175341  0.33933701]
[ 0.31980487  0.30982844  0.37036668]
[ 0.41375208  0.34639338  0.23985454]
[ 0.36916022  0.42205857  0.20878121]
[ 0.38792322  0.29737495  0.31470182]
[ 0.28501557  0.33200044  0.382984  ]
[ 0.29016413  0.36385007  0.3459858 ]
[ 0.32938116  0.34192338  0.32869546]
[ 0.35108971  0.30940691  0.33950337]
[ 0.36302523  0.3325479   0.30442686]
[ 0.34151407  0.25932795  0.39915798]
[ 0.43692036  0.30189597  0.26118367]
[ 0.31930026  0.33054225  0.35015749]
[ 0.33602996  0.29799794  0.3659721 ]
[ 0.29711624  0.30519323  0.39769053]
Term: skunk
New topic distribution: [ 4.  1.  3.]
Term: can
New topic distribution: [ 1.  4.  4.]
